In [5]:
%load_ext autoreload
%autoreload 2

from pprint import pprint
import sys
import os, json, re, requests
import ckanapi.errors
from ckanapi.errors import NotFound
import pandas as pd
import random

from pathlib import Path
import json

from ckan_migration import Migrate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ORIGINAL_CKAN_URL = 'http://staging.basedosdados.org'

In [3]:
def get_package_show(package):
    api_url = ORIGINAL_CKAN_URL + f'/api/3/action/package_show?id={package}'
    print(api_url)

    return requests.get(api_url, verify=False).json()['result']

def download_packages():
    api_url = ORIGINAL_CKAN_URL + '/api/3/action/package_list'
    packages = requests.get(api_url, verify=False).json()['result']

    for p in packages:
        json.dump(get_package_show(p), open(f'packages/{p}', 'w'))


In [4]:
download_packages()

In [4]:

datas_list = ['2008-2019', '1987-88, 1995-96, 2002-03, 2008-09', 'Atual',
             '-', '2019', '1978, 1992-2019'
             ]
def findkeys(node, kv):
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
                yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

In [6]:
packages_left = [str(p) for p in Path('packages').glob('*')]

for p in Path('packages').glob('*'):
    package = json.loads(p.open().read())
    a = Migrate(package)
    a.migrate()
    val = a.validate()
#     Para migrar o staging usar update_staging, testar com apenas um package
    print(p.name)
    a.update_staging()
#     break
    
#     if isinstance(val, list):
#         print(val)
#         print(a.package_dict)
#         key_errors = list(set([d['loc'][3] for d in val]))
#         _dict_errors = {key:list(findkeys(a.package_dict, key)) for key in key_errors}
        
#         print('package: ',p,'\n')
#         print('package_errors: ',_dict_errors,'\n')
#         print('packages left: ',len(packages_left))

#         break
    
#     packages_left.remove(f"packages/{p.name}")
#     break

br_abrinq_oca
letourdataset
a-global-database-on-central-banks-monetary-responses-to-covid-19
marinetraffic
operabase
covid-19-preprints
cdc-s-social-vulnerability-index-svi
world-sea-temperatures
passageiros-no-titanic
br-ibge-pib
u-s-survey-of-doctorate-recipients-sdr
world-atlas-of-language-structures-wals
br-ibge-amc
nobel-laureates-1901-present
smithsonian-open-access
global-carbon-atlas
occupied-palestinian-territory
genomic-data-commons-gdc
the-microbe-directory
atlas-da-violencia
macro-financial-dataset
large-logo-dataset-lld
uniform-crime-reporting-ucr-program-data-arrests-by-age-sex-and-race
diario-oficial-do-estado-do-distrito-federal
pesquisa-de-opiniao-american-trends-panel
oecd-tax-database
global-attitudes-on-abortion
prestadores-de-servicos-turisticos-cadastur
human-mortality-database-hmd
territorial-self-governance-dataset-terrgo
museum-data-files-mdf
u-s-post-offices
br_camara_atividade_legislativa
comtrade
public-utility-data-liberation-project-pudl
every-politician


In [192]:
#OLD LOG

# for p in Path('packages').glob('*'):
#     if p.name not in validated_list:
#         package = json.loads(p.open().read())
#         a = Migrate(package)
#         a.migrate()
#         val = a.validate()
#         if isinstance(val, list):
#             print(p.name)
#             print(val, '\n')
#             print(a.package_dict, '\n')
            
#             break
#         validated_list.append(p.name)